In [1]:
# ! pip install mlxtend
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

sales_fact_1998_ori_df = pd.read_csv("./hw1_data/sales_fact_1998.csv")
sales_fact_dec_1998_ori_df = pd.read_csv("./hw1_data/sales_fact_dec_1998.csv")
product_ori_df = pd.read_csv("./hw1_data/product.csv")
customer_ori_df = pd.read_csv("./hw1_data/customer.csv")
transaction_ori_df = pd.concat([sales_fact_1998_ori_df, sales_fact_dec_1998_ori_df]).reset_index() # concatenate two datasets

time: 283 ms (started: 2022-01-17 07:55:29 +00:00)


# Q1

In [2]:
transaction_df = transaction_ori_df.copy().drop(['promotion_id', 'unit_sales', 'index', 'store_sales', 'store_cost'], axis=1) # we only need product_id, time_id, customer_id, store_id
transaction_df['product_id'] = transaction_ori_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0]) # change product_id to product name
transaction_df = pd.concat([transaction_df, pd.get_dummies(transaction_df['product_id'])], axis=1).drop(['product_id'],axis=1) # onehot encoding
transaction_df = transaction_df.drop_duplicates()
transaction_df = transaction_df.groupby(['time_id', 'customer_id', 'store_id']).sum()
display(transaction_df)

ADJ Rosy Sunglasses  Akron City Map  \
time_id customer_id store_id                                        
732     534         10                          0               0   
        1013        10                          0               0   
        1325        10                          0               0   
        1425        10                          0               0   
        1550        10                          0               0   
...                                           ...             ...   
1095    9559        10                          0               0   
        9696        11                          0               0   
        9725        10                          0               0   
        9743        10                          0               0   
        10095       10                          0               0   

                              Akron Eyeglass Screwdriver  \
time_id customer_id store_id                               
732     534         10                                 0   
        1013        10                                 0   
        1325        10                                 0   
        1425        10                                 0   
        1550        10                                 0   
...                                                  ...   
1095    9559        10                                 0   
        9696        11                                 0   
        9725        10                                 0   
        9743        10                                 0   
        10095       10                                 0   

                              American Beef Bologna  \
time_id customer_id store_id                          
732     534         10                            0   
        1013        10                            0   
        1325        10                            0   
        1425        10                            0   
        1550        10                            0   
...                                             ...   
1095    9559        10                            0   
        9696        11                            0   
        9725        10                            0   
        9743        10                            0   
        10095       10                            0   

                              American Chicken Hot Dogs  American Cole Slaw  \
time_id customer_id store_id                                                  
732     534         10                                0                   0   
        1013        10                                0                   0   
        1325        10                                0                   0   
        1425        10                                0                   0   
        1550        10                                0                   0   
...                                                 ...                 ...   
1095    9559        10                                0                   0   
        9696        11                                0                   0   
        9725        10                                0                   0   
        9743        10                                0                   0   
        10095       10                                0                   0   

                              American Corned Beef  \
time_id customer_id store_id                         
732     534         10                           0   
        1013        10                           0   
        1325        10                           0   
        1425        10                           0   
        1550        10                           0   
...                                            ...   
1095    9559        10                           0   
        9696        11                           0   
        9725        10                           0   
        9743 

time: 34.6 s (started: 2022-01-17 07:55:29 +00:00)


In [3]:
# an_col = [col for col in transaction_df.columns if (set(transaction_df[col].unique().tolist()) != {0,1})]
# duplicate_ind = []
# for ind in transaction_df.loc[(transaction_df==2).any(axis=1),an_col].index:
#     cond = np.logical_and(np.logical_and(transaction_ori_df['time_id']==ind[0], transaction_ori_df['customer_id']==ind[1]), transaction_ori_df['store_id']==ind[2])
#     tmp = transaction_ori_df.loc[cond , 'product_id'].duplicated()
#     duplicate_ind.append(tmp.loc[tmp==1].index.values[0])
    
# duplicate_ind

time: 317 µs (started: 2022-01-17 07:56:04 +00:00)


In [4]:
transac_apriori_freq_itemset = apriori(transaction_df, min_support=0.0001, use_colnames=True, low_memory=True) # use apriori to find frequent itemset
transac_apriori_rules = association_rules(transac_apriori_freq_itemset, metric='confidence', min_threshold=0.9) # use frequent itemset to find rules
transac_apriori_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False).to_csv('apriori_product.csv')
display(transac_apriori_freq_itemset.sort_values(by = 'support', ascending=False), 
        transac_apriori_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False))

,support,itemsets
1084,0.004148,(Moms Roasted Chicken)
746,0.004095,(Great English Muffins)
1389,0.003989,(Super Chunky Peanut Butter)
748,0.003963,(Great Pumpernickel Bread)
1117,0.003936,(Nationeel Dried Apples)
...,...,...
9085,0.000106,"(Even Better 2% Milk, Toucan Canned Mixed Fruit)"
9086,0.000106,"(Even Better 2% Milk, Walrus Light Wine)"
9088,0.000106,"(Even Better Blueberry Yogurt, Hermanos Fancy ..."
9090,0.000106,"(Imagine Lime Popsicles, Even Better Blueberry..."


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
381,"(Imagine Low Fat French Fries, CDR Hot Chocolate)",(Quick Extra Lean Hamburger),0.000185,0.003117,0.000185,1.0,320.771186,0.000184,inf
702,"(Just Right Canned Yams, Hilltop 200 MG Acetom...",(Faux Products HCL Nasal Spray),0.000159,0.002959,0.000159,1.0,337.955357,0.000158,inf
325,"(Bravo Fancy Canned Anchovies, Booker Low Fat ...",(High Top Oranges),0.000159,0.003197,0.000159,1.0,312.818182,0.000158,inf
598,"(Plato French Roast Coffee, High Quality Sciss...",(Dollar Monthly Sports Magazine),0.000159,0.003197,0.000159,1.0,312.818182,0.000158,inf
550,"(Cormorant Scented Toilet Tissue, Hilltop 200 ...",(Horatio No Salt Popcorn),0.000159,0.003329,0.000159,1.0,300.404762,0.000158,inf
...,...,...,...,...,...,...,...,...,...
1427,"(Dual City Lox, Cutting Edge Roasted Chicken, ...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
759,"(Tell Tale Lemons, High Quality Paper Towels)",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
1119,"(Hilltop Silky Smooth Hair Conditioner, Big Ti...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
1116,"(Big Time Popsicles, Consolidated Mint Mouthwa...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf


time: 3.62 s (started: 2022-01-17 07:56:04 +00:00)


# Q2

In [5]:
transac_fp_freq_itemset = fpgrowth(transaction_df, min_support=0.0001, use_colnames=True) # use fp-growth to find frequent_itemset
transac_fp_rules = association_rules(transac_fp_freq_itemset, metric='confidence', min_threshold=0.9) # find rules
transac_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False).to_csv('fp_product.csv')
display(transac_fp_freq_itemset.sort_values(by = 'support', ascending=False), 
        transac_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False))

,support,itemsets
383,0.004148,(Moms Roasted Chicken)
454,0.004095,(Great English Muffins)
595,0.003989,(Super Chunky Peanut Butter)
1509,0.003963,(Great Pumpernickel Bread)
1032,0.003936,(Carrington Ice Cream)
...,...,...
7334,0.000106,"(Best Choice Dried Apples, Genteel Extra Lean ..."
7335,0.000106,"(Pleasant Canned Tuna in Water, Best Choice Dr..."
7336,0.000106,"(Best Choice Dried Apples, Nationeel Mini Donuts)"
7337,0.000106,"(Best Choice Dried Apples, Ebony Broccoli)"


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
805,"(Imagine Low Fat French Fries, CDR Hot Chocolate)",(Quick Extra Lean Hamburger),0.000185,0.003117,0.000185,1.0,320.771186,0.000184,inf
309,"(Just Right Canned Yams, Hilltop 200 MG Acetom...",(Faux Products HCL Nasal Spray),0.000159,0.002959,0.000159,1.0,337.955357,0.000158,inf
1249,"(Plato French Roast Coffee, High Quality Sciss...",(Dollar Monthly Sports Magazine),0.000159,0.003197,0.000159,1.0,312.818182,0.000158,inf
1534,"(Bravo Fancy Canned Anchovies, Booker Low Fat ...",(High Top Oranges),0.000159,0.003197,0.000159,1.0,312.818182,0.000158,inf
1190,"(Cormorant Scented Toilet Tissue, Hilltop 200 ...",(Horatio No Salt Popcorn),0.000159,0.003329,0.000159,1.0,300.404762,0.000158,inf
...,...,...,...,...,...,...,...,...,...
1051,"(Hilltop Silky Smooth Hair Conditioner, Big Ti...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
1048,"(Big Time Popsicles, Consolidated Mint Mouthwa...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
453,"(Tell Tale Lemons, High Quality Paper Towels)",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
448,"(Dual City Lox, Cutting Edge Roasted Chicken, ...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf


time: 1.37 s (started: 2022-01-17 07:56:07 +00:00)


# Q3

In [6]:
cus_attr = ["state_province", "yearly_income", "gender", "total_children", "num_children_at_home", "education", "occupation", "houseowner", "num_cars_owned"]
customer_attr_df = customer_ori_df.loc[:, cus_attr] # we choose the wanted columns
discret_cus_df = customer_attr_df.select_dtypes(include='int').apply(lambda x:pd.cut(x, x.max())) # eqivalent frequency cut the numerical attributes
customer_cat_df = pd.concat([customer_attr_df.select_dtypes(exclude='int'), discret_cus_df], axis=1) # concatenate categorical attributes
customer_oh_cat_df = pd.get_dummies(customer_cat_df)
display(customer_oh_cat_df)

,state_province_BC,state_province_CA,state_province_DF,state_province_Guerrero,state_province_Jalisco,state_province_Mexico,state_province_OR,state_province_Oaxaca,state_province_Sinaloa,state_province_Veracruz,...,"total_children_(4.0, 5.0]","num_children_at_home_(-0.005, 1.0]","num_children_at_home_(1.0, 2.0]","num_children_at_home_(2.0, 3.0]","num_children_at_home_(3.0, 4.0]","num_children_at_home_(4.0, 5.0]","num_cars_owned_(-0.004, 1.0]","num_cars_owned_(1.0, 2.0]","num_cars_owned_(2.0, 3.0]","num_cars_owned_(3.0, 4.0]"
0,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10276,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
10277,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
10278,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
10279,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,0


time: 28.1 ms (started: 2022-01-17 07:56:09 +00:00)


In [7]:
cus_fp_freq_itemset = fpgrowth(customer_oh_cat_df, min_support=0.05, use_colnames=True)
cus_fp_rules = association_rules(cus_fp_freq_itemset, metric='confidence', min_threshold=0.9)
cus_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False).to_csv('fp_customer.csv')
display(cus_fp_freq_itemset.sort_values(by = 'support', ascending=False), 
        cus_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False))

,support,itemsets
8,0.773466,"(num_children_at_home_(-0.005, 1.0])"
0,0.601790,(houseowner_Y)
9,0.504231,(gender_M)
1,0.495769,(gender_F)
33,0.427877,"(num_children_at_home_(-0.005, 1.0], houseowne..."
...,...,...
180,0.050190,"(num_children_at_home_(-0.005, 1.0], num_cars_..."
531,0.050092,"(gender_F, yearly_income_$10K - $30K, occupati..."
545,0.050092,"(total_children_(-0.005, 1.0], yearly_income_$..."
547,0.050092,"(yearly_income_$10K - $30K, total_children_(-0..."


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,"(total_children_(-0.005, 1.0])","(num_children_at_home_(-0.005, 1.0])",0.297928,0.773466,0.297928,1.000000,1.292882,0.067491,inf
15,"(total_children_(-0.005, 1.0], houseowner_Y)","(num_children_at_home_(-0.005, 1.0])",0.178290,0.773466,0.178290,1.000000,1.292882,0.040389,inf
5,"(total_children_(-0.005, 1.0], gender_M)","(num_children_at_home_(-0.005, 1.0])",0.154654,0.773466,0.154654,1.000000,1.292882,0.035034,inf
16,"(total_children_(-0.005, 1.0], gender_F)","(num_children_at_home_(-0.005, 1.0])",0.143274,0.773466,0.143274,1.000000,1.292882,0.032456,inf
20,"(state_province_CA, total_children_(-0.005, 1.0])","(num_children_at_home_(-0.005, 1.0])",0.124696,0.773466,0.124696,1.000000,1.292882,0.028248,inf
...,...,...,...,...,...,...,...,...,...
2,"(education_High School Degree, occupation_Skil...",(yearly_income_$30K - $50K),0.056707,0.323607,0.051260,0.903945,2.793345,0.032909,7.041738
38,"(num_children_at_home_(-0.005, 1.0], education...",(yearly_income_$10K - $30K),0.065752,0.216127,0.059333,0.902367,4.175173,0.045122,8.028761
0,"(gender_F, yearly_income_$30K - $50K, houseown...","(num_children_at_home_(-0.005, 1.0])",0.066725,0.773466,0.060208,0.902332,1.166610,0.008599,2.319442
91,"(occupation_Manual, education_High School Degree)",(yearly_income_$30K - $50K),0.113899,0.323607,0.102714,0.901793,2.786696,0.065855,6.887449


time: 173 ms (started: 2022-01-17 07:56:09 +00:00)


# Q4

In [8]:
transaction_df = transaction_ori_df.copy().drop(['time_id', 'store_id', 'promotion_id', 'unit_sales', 'index', 'store_sales', 'store_cost'], axis=1) # we only need product_id, customer_id
transaction_df['product_id'] = transaction_ori_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0]) # change product_id to product name
customer_cat_cid_df = pd.concat([customer_cat_df, customer_ori_df['customer_id']], axis=1)
transac_cus_df = pd.merge(transaction_df, customer_cat_cid_df, on='customer_id')
transac_cus_oh_df = pd.get_dummies(transac_cus_df.drop(['customer_id'], axis=1))
display(transac_cus_oh_df)

,product_id_ADJ Rosy Sunglasses,product_id_Akron City Map,product_id_Akron Eyeglass Screwdriver,product_id_American Beef Bologna,product_id_American Chicken Hot Dogs,product_id_American Cole Slaw,product_id_American Corned Beef,product_id_American Foot-Long Hot Dogs,product_id_American Low Fat Bologna,product_id_American Low Fat Cole Slaw,...,"total_children_(4.0, 5.0]","num_children_at_home_(-0.005, 1.0]","num_children_at_home_(1.0, 2.0]","num_children_at_home_(2.0, 3.0]","num_children_at_home_(3.0, 4.0]","num_children_at_home_(4.0, 5.0]","num_cars_owned_(-0.004, 1.0]","num_cars_owned_(1.0, 2.0]","num_cars_owned_(2.0, 3.0]","num_cars_owned_(3.0, 4.0]"
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182878,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
182879,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
182880,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
182881,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


time: 28.4 s (started: 2022-01-17 07:56:09 +00:00)


In [9]:
transac_cus_fp_freq_itemset = fpgrowth(transac_cus_oh_df, min_support=0.0001, max_len=2, use_colnames=True) # use fp-growth to find frequent_itemset
transac_cus_fp_rules = association_rules(transac_cus_fp_freq_itemset, metric='confidence', min_threshold=0.001) # find rules
filter_ind = transac_cus_fp_rules['consequents'].apply(lambda x:x.isdisjoint(customer_oh_cat_df.keys().tolist())==True).tolist() # choose the consequents consist only of product
transac_cus_filtered_fp_rules = transac_cus_fp_rules.iloc[filter_ind,:] # choose the consequents consist only of product
transac_cus_filtered_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False).to_csv('fp_quanti_rules.csv')
display(transac_cus_fp_freq_itemset.sort_values(by = 'support', ascending=False), 
        transac_cus_filtered_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False))

,support,itemsets
0,0.772439,"(num_children_at_home_(-0.005, 1.0])"
1,0.604534,(houseowner_Y)
2,0.510436,(gender_F)
167,0.489564,(gender_M)
1607,0.426688,"(num_children_at_home_(-0.005, 1.0], houseowne..."
...,...,...
3826,0.000104,"(total_children_(2.0, 3.0], product_id_Carring..."
18934,0.000104,"(product_id_Best Wheat Puffs, yearly_income_$7..."
18937,0.000104,"(product_id_Best Wheat Puffs, num_children_at_..."
18945,0.000104,"(state_province_OR, product_id_Best Wheat Puffs)"


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
18752,"(num_children_at_home_(2.0, 3.0])",(product_id_Hermanos Peaches),0.070105,0.000760,0.000104,0.001482,1.949801,0.000051,1.000723
18044,"(num_children_at_home_(2.0, 3.0])",(product_id_Best Wheat Puffs),0.070105,0.000776,0.000104,0.001482,1.908608,0.000049,1.000707
2611,(state_province_Yucatan),(product_id_Better Noodle Soup),0.071160,0.000755,0.000104,0.001460,1.934804,0.000050,1.000706
4878,(state_province_Yucatan),(product_id_High Top New Potatos),0.071160,0.000771,0.000104,0.001460,1.893638,0.000049,1.000690
7945,(education_Partial College),(product_id_Nationeel Grape Fruit Roll),0.093344,0.000705,0.000131,0.001406,1.993131,0.000065,1.000702
...,...,...,...,...,...,...,...,...,...
31772,(yearly_income_$50K - $70K),(product_id_Tri-State Honey Dew),0.169387,0.000705,0.000170,0.001001,1.418704,0.000050,1.000296
36316,(yearly_income_$50K - $70K),(product_id_Ebony Corn on the Cob),0.169387,0.000705,0.000170,0.001001,1.418704,0.000050,1.000296
34494,(yearly_income_$50K - $70K),(product_id_Pleasant Canned Tuna in Water),0.169387,0.000716,0.000170,0.001001,1.397045,0.000048,1.000285
2564,(yearly_income_$50K - $70K),(product_id_Framton City Map),0.169387,0.000749,0.000170,0.001001,1.335860,0.000043,1.000252


time: 7.86 s (started: 2022-01-17 07:56:37 +00:00)


# Q5

In [10]:
transaction_before_dec_df = sales_fact_1998_ori_df.copy().drop(['promotion_id', 'unit_sales', 'store_sales', 'store_cost'], axis=1) # we only need product_id, time_id, customer_id, store_id
transaction_dec_df = sales_fact_dec_1998_ori_df.copy().drop(['promotion_id', 'unit_sales', 'store_sales', 'store_cost'], axis=1) # we only need product_id, time_id, customer_id, store_id
transaction_before_dec_df['product_id'] = transaction_before_dec_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0]) # change product_id to product name
transaction_dec_df['product_id'] = transaction_dec_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0]) # change product_id to product name
transaction_before_dec_df = pd.concat([transaction_before_dec_df, pd.get_dummies(transaction_before_dec_df['product_id'])], axis=1).drop(['product_id'],axis=1) # onehot encoding
transaction_dec_df = pd.concat([transaction_dec_df, pd.get_dummies(transaction_dec_df['product_id'])], axis=1).drop(['product_id'],axis=1) # onehot encoding
transaction_before_dec_df = transaction_before_dec_df.drop_duplicates()
transaction_dec_df = transaction_dec_df.drop_duplicates()
transaction_before_dec_df = transaction_before_dec_df.groupby(['time_id', 'customer_id', 'store_id']).sum()
transaction_dec_df = transaction_dec_df.groupby(['time_id', 'customer_id', 'store_id']).sum()
display(transaction_dec_df, transaction_before_dec_df)

ADJ Rosy Sunglasses  Akron City Map  \
time_id customer_id store_id                                        
1066    217         13                          0               0   
        465         13                          0               0   
        484         13                          0               0   
        619         13                          0               0   
        767         4                           0               0   
...                                           ...             ...   
1095    9559        10                          0               0   
        9696        11                          0               0   
        9725        10                          0               0   
        9743        10                          0               0   
        10095       10                          0               0   

                              Akron Eyeglass Screwdriver  \
time_id customer_id store_id                               
1066    217         13                                 0   
        465         13                                 0   
        484         13                                 0   
        619         13                                 0   
        767         4                                  0   
...                                                  ...   
1095    9559        10                                 0   
        9696        11                                 0   
        9725        10                                 0   
        9743        10                                 0   
        10095       10                                 0   

                              American Beef Bologna  \
time_id customer_id store_id                          
1066    217         13                            0   
        465         13                            0   
        484         13                            0   
        619         13                            0   
        767         4                             0   
...                                             ...   
1095    9559        10                            0   
        9696        11                            0   
        9725        10                            0   
        9743        10                            0   
        10095       10                            0   

                              American Chicken Hot Dogs  American Cole Slaw  \
time_id customer_id store_id                                                  
1066    217         13                                0                   0   
        465         13                                0                   0   
        484         13                                0                   0   
        619         13                                0                   0   
        767         4                                 0                   0   
...                                                 ...                 ...   
1095    9559        10                                0                   0   
        9696        11                                0                   0   
        9725        10                                0                   0   
        9743        10                                0                   0   
        10095       10                                0                   0   

                              American Corned Beef  \
time_id customer_id store_id                         
1066    217         13                           0   
        465         13                           0   
        484         13                           0   
        619         13                           0   
        767         4                            0   
...                                            ...   
1095    9559        10                           0   
        9696        11                           0   
        9725        10                           0   
        9743 

ADJ Rosy Sunglasses  Akron City Map  \
time_id customer_id store_id                                        
732     534         10                          0               0   
        1013        10                          0               0   
        1325        10                          0               0   
        1425        10                          0               0   
        1550        10                          0               0   
...                                           ...             ...   
1065    9510        16                          0               0   
        9728        16                          0               0   
        9743        10                          0               0   
        9897        16                          0               0   
        10260       11                          0               0   

                              Akron Eyeglass Screwdriver  \
time_id customer_id store_id                               
732     534         10                                 0   
        1013        10                                 0   
        1325        10                                 0   
        1425        10                                 0   
        1550        10                                 0   
...                                                  ...   
1065    9510        16                                 0   
        9728        16                                 0   
        9743        10                                 0   
        9897        16                                 0   
        10260       11                                 0   

                              American Beef Bologna  \
time_id customer_id store_id                          
732     534         10                            0   
        1013        10                            0   
        1325        10                            0   
        1425        10                            0   
        1550        10                            0   
...                                             ...   
1065    9510        16                            0   
        9728        16                            0   
        9743        10                            0   
        9897        16                            0   
        10260       11                            0   

                              American Chicken Hot Dogs  American Cole Slaw  \
time_id customer_id store_id                                                  
732     534         10                                0                   0   
        1013        10                                0                   0   
        1325        10                                0                   0   
        1425        10                                0                   0   
        1550        10                                0                   0   
...                                                 ...                 ...   
1065    9510        16                                0                   0   
        9728        16                                0                   0   
        9743        10                                0                   0   
        9897        16                                0                   0   
        10260       11                                0                   0   

                              American Corned Beef  \
time_id customer_id store_id                         
732     534         10                           0   
        1013        10                           0   
        1325        10                           0   
        1425        10                           0   
        1550        10                           0   
...                                            ...   
1065    9510        16                           0   
        9728        16                           0   
        9743        10                           0   
        9897 

time: 34.3 s (started: 2022-01-17 07:56:45 +00:00)


In [11]:
transac_before_dec_freq_itemset = fpgrowth(transaction_before_dec_df, min_support=0.0001, use_colnames=True) # use fp-growth to find frequent_itemset
transac_before_dec_rules = association_rules(transac_before_dec_freq_itemset, metric='confidence', min_threshold=0.05) # find rules
transac_dec_freq_itemset = fpgrowth(transaction_dec_df, min_support=0.0003, use_colnames=True) # use fp-growth to find frequent_itemset
transac_dec_rules = association_rules(transac_dec_freq_itemset, metric='confidence', min_threshold=0.05) # find rules
display(transac_before_dec_rules, transac_dec_rules) 
display(transac_dec_freq_itemset, transac_before_dec_freq_itemset)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Big Time Apple Cinnamon Waffles),(Toucan Canned Mixed Fruit),0.003434,0.003552,0.000205,0.059829,16.846083,0.000193,1.059859
1,(Toucan Canned Mixed Fruit),(Big Time Apple Cinnamon Waffles),0.003552,0.003434,0.000205,0.057851,16.846083,0.000193,1.057759
2,(Big Time Apple Cinnamon Waffles),(High Top Corn on the Cob),0.003434,0.003610,0.000176,0.051282,14.204711,0.000164,1.050249
3,(Bravo Fancy Canned Anchovies),(High Top Oranges),0.003610,0.003434,0.000235,0.065041,18.939615,0.000222,1.065892
4,(High Top Oranges),(Bravo Fancy Canned Anchovies),0.003434,0.003610,0.000235,0.068376,18.939615,0.000222,1.069519
...,...,...,...,...,...,...,...,...,...
6416,"(Imagine Frozen Chicken Thighs, Excellent Diet...","(PigTail Ice Cream Sandwich, Kiwi Lox)",0.000176,0.000117,0.000117,0.666667,5678.333333,0.000117,2.999648
6417,"(Imagine Frozen Chicken Thighs, Kiwi Lox)","(PigTail Ice Cream Sandwich, Excellent Diet Soda)",0.000117,0.000147,0.000117,1.000000,6814.000000,0.000117,inf
6418,"(PigTail Ice Cream Sandwich, Excellent Diet Soda)","(Imagine Frozen Chicken Thighs, Kiwi Lox)",0.000147,0.000117,0.000117,0.800000,6814.000000,0.000117,4.999413
6419,"(PigTail Ice Cream Sandwich, Kiwi Lox)","(Imagine Frozen Chicken Thighs, Excellent Diet...",0.000117,0.000176,0.000117,1.000000,5678.333333,0.000117,inf


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Toucan Canned Peaches),(Cutting Edge Sliced Turkey),0.003438,0.003438,0.000529,0.153846,44.745562,0.000517,1.177755
1,(Cutting Edge Sliced Turkey),(Toucan Canned Peaches),0.003438,0.003438,0.000529,0.153846,44.745562,0.000517,1.177755
2,(Toucan Canned Peaches),(High Top Asparagus),0.003438,0.003967,0.000529,0.153846,38.779487,0.000515,1.177130
3,(High Top Asparagus),(Toucan Canned Peaches),0.003967,0.003438,0.000529,0.133333,38.779487,0.000515,1.149879
4,(Toucan Canned Peaches),(Bravo Creamed Corn),0.003438,0.003703,0.000529,0.153846,41.549451,0.000516,1.177442
...,...,...,...,...,...,...,...,...,...
12955,(Washington Orange Juice),"(Best Choice Sesame Crackers, Hermanos Plums, ...",0.003967,0.000529,0.000529,0.133333,252.066667,0.000527,1.153236
12956,(Sunset 75 Watt Lightbulb),"(Best Choice Sesame Crackers, Hermanos Plums, ...",0.005554,0.000529,0.000529,0.095238,180.047619,0.000526,1.104679
12957,(Best Choice Sesame Crackers),"(Hermanos Plums, Consolidated Childrens Cold R...",0.002909,0.000529,0.000529,0.181818,343.727273,0.000527,1.221576
12958,(Applause Canned Mixed Fruit),(Gorilla Low Fat Sour Cream),0.002116,0.002645,0.000529,0.250000,94.525000,0.000523,1.329807


,support,itemsets
0,0.003438,(Cutting Edge Sliced Turkey)
1,0.003438,(Toucan Canned Peaches)
2,0.001851,(Hilltop Childrens Aspirin)
3,0.001587,(Horatio Low Fat Chips)
4,0.004232,(Blue Label Turkey Noodle Soup)
...,...,...
4530,0.000529,"(Hermanos Plums, Best Choice Sesame Crackers, ..."
4531,0.000529,"(Best Choice Sesame Crackers, Hermanos Plums, ..."
4532,0.000529,"(Consolidated Childrens Cold Remedy, Best Choi..."
4533,0.000529,"(Hermanos Plums, Consolidated Childrens Cold R..."


,support,itemsets
0,0.003552,(Toucan Canned Mixed Fruit)
1,0.003434,(Big Time Apple Cinnamon Waffles)
2,0.003434,(High Top Oranges)
3,0.003258,(BBB Best Canola Oil)
4,0.003199,(Excellent Apple Drink)
...,...,...
12515,0.000117,"(PigTail Ice Cream Sandwich, Imagine Ice Cream..."
12516,0.000117,"(PigTail Ice Cream Sandwich, Excellent Diet So..."
12517,0.000117,"(Imagine Frozen Chicken Thighs, PigTail Ice Cr..."
12518,0.000117,"(Imagine Frozen Chicken Thighs, PigTail Ice Cr..."


time: 1.55 s (started: 2022-01-17 07:57:20 +00:00)


In [12]:
common_rules = pd.merge(transac_before_dec_rules, transac_dec_rules, on=["antecedents", "consequents"], suffixes=('_before', '_dec'))
common_rules = common_rules.sort_values(by = ['confidence_before', 'confidence_dec', 
                                              'support_before', 'support_dec', 
                                              'lift_before', 'lift_dec', 
                                              'leverage_before', 'leverage_dec', 
                                              'antecedent support_before', 'antecedent support_dec', 
                                              'consequent support_before', 'consequent support_dec', 
                                              'antecedents'], ascending=False)
common_rules.to_csv('dec_common_rules.csv')
display(common_rules)

,antecedents,consequents,antecedent support_before,consequent support_before,support_before,confidence_before,lift_before,leverage_before,conviction_before,antecedent support_dec,consequent support_dec,support_dec,confidence_dec,lift_dec,leverage_dec,conviction_dec
27,(Big Time Popsicles),(Hilltop Silky Smooth Hair Conditioner),0.003170,0.003669,0.000264,0.083333,22.713333,0.000253,1.086907,0.002645,0.004761,0.000529,0.200000,42.011111,0.000516,1.244049
38,(Hermanos Almonds),(Tell Tale Fresh Lima Beans),0.003405,0.003757,0.000264,0.077586,20.651266,0.000251,1.080039,0.004761,0.004496,0.000529,0.111111,24.712418,0.000508,1.119942
13,(Medalist Spaghetti),(Excellent Mango Drink),0.003141,0.003493,0.000235,0.074766,21.405796,0.000224,1.077033,0.002380,0.004761,0.000529,0.222222,46.679012,0.000518,1.279593
26,(Hilltop Silky Smooth Hair Conditioner),(Big Time Popsicles),0.003669,0.003170,0.000264,0.072000,22.713333,0.000253,1.074170,0.004761,0.002645,0.000529,0.111111,42.011111,0.000516,1.122025
5,(Best Choice Frosted Donuts),(BBB Best Apple Butter),0.003287,0.003552,0.000235,0.071429,20.112161,0.000223,1.073098,0.002909,0.002909,0.000529,0.181818,62.495868,0.000520,1.218666
39,(Tell Tale Fresh Lima Beans),(Hermanos Almonds),0.003757,0.003405,0.000264,0.070312,20.651266,0.000251,1.071968,0.004496,0.004761,0.000529,0.117647,24.712418,0.000508,1.127938
14,(Excellent Mango Drink),(Medalist Spaghetti),0.003493,0.003141,0.000235,0.067227,21.405796,0.000224,1.068705,0.004761,0.002380,0.000529,0.111111,46.679012,0.000518,1.122322
32,(Best Choice Frosted Cookies),(Fabulous Strawberry Drink),0.002642,0.003522,0.000176,0.066667,18.927778,0.000167,1.067655,0.003174,0.003967,0.000529,0.166667,42.011111,0.000516,1.195239
6,(BBB Best Apple Butter),(Best Choice Frosted Donuts),0.003552,0.003287,0.000235,0.066116,20.112161,0.000223,1.067276,0.002909,0.002909,0.000529,0.181818,62.495868,0.000520,1.218666
40,(Fast Low Fat Cookies),(Cormorant Paper Plates),0.002700,0.003434,0.000176,0.065217,18.991081,0.000167,1.066094,0.005025,0.003174,0.000529,0.105263,33.166667,0.000513,1.114100


time: 41.4 ms (started: 2022-01-17 07:57:21 +00:00)


In [13]:
dec_only_rules = pd.merge(transac_dec_rules.loc[:, ['antecedents', 'consequents']], transac_before_dec_rules.loc[:, ['antecedents', 'consequents']], on=['antecedents', 'consequents'], how='left', indicator = True).loc[lambda x : x['_merge']!='both']
before_dec_only_rules = pd.merge(transac_dec_rules.loc[:,['antecedents', 'consequents']], transac_before_dec_rules.loc[:, ['antecedents', 'consequents']], on=['antecedents', 'consequents'], how='right', indicator = True).loc[lambda x : x['_merge']!='both']
display(dec_only_rules, before_dec_only_rules)

,antecedents,consequents,_merge
0,(Toucan Canned Peaches),(Cutting Edge Sliced Turkey),left_only
1,(Cutting Edge Sliced Turkey),(Toucan Canned Peaches),left_only
2,(Toucan Canned Peaches),(High Top Asparagus),left_only
3,(High Top Asparagus),(Toucan Canned Peaches),left_only
4,(Toucan Canned Peaches),(Bravo Creamed Corn),left_only
...,...,...,...
12955,(Washington Orange Juice),"(Best Choice Sesame Crackers, Hermanos Plums, ...",left_only
12956,(Sunset 75 Watt Lightbulb),"(Best Choice Sesame Crackers, Hermanos Plums, ...",left_only
12957,(Best Choice Sesame Crackers),"(Hermanos Plums, Consolidated Childrens Cold R...",left_only
12958,(Applause Canned Mixed Fruit),(Gorilla Low Fat Sour Cream),left_only


,antecedents,consequents,_merge
0,(Big Time Apple Cinnamon Waffles),(Toucan Canned Mixed Fruit),right_only
1,(Toucan Canned Mixed Fruit),(Big Time Apple Cinnamon Waffles),right_only
2,(Big Time Apple Cinnamon Waffles),(High Top Corn on the Cob),right_only
3,(Bravo Fancy Canned Anchovies),(High Top Oranges),right_only
4,(High Top Oranges),(Bravo Fancy Canned Anchovies),right_only
...,...,...,...
6416,"(Imagine Frozen Chicken Thighs, Excellent Diet...","(PigTail Ice Cream Sandwich, Kiwi Lox)",right_only
6417,"(Imagine Frozen Chicken Thighs, Kiwi Lox)","(PigTail Ice Cream Sandwich, Excellent Diet Soda)",right_only
6418,"(PigTail Ice Cream Sandwich, Excellent Diet Soda)","(Imagine Frozen Chicken Thighs, Kiwi Lox)",right_only
6419,"(PigTail Ice Cream Sandwich, Kiwi Lox)","(Imagine Frozen Chicken Thighs, Excellent Diet...",right_only


time: 34.7 ms (started: 2022-01-17 07:57:21 +00:00)


In [14]:
transac_dec_freq_itemset = fpgrowth(transaction_dec_df, min_support=0.0001, use_colnames=True) # use fp-growth to find frequent_itemset
transac_dec_freq_itemset 

,support,itemsets
0,0.003438,(Cutting Edge Sliced Turkey)
1,0.003438,(Toucan Canned Peaches)
2,0.001851,(Hilltop Childrens Aspirin)
3,0.001587,(Horatio Low Fat Chips)
4,0.004232,(Blue Label Turkey Noodle Soup)
...,...,...
26532298,0.000264,"(Landslide Creamy Peanut Butter, Denny Plastic..."
26532299,0.000264,"(Imagine Frozen Broccoli, Denny Plastic Knives..."
26532300,0.000264,"(Imagine Frozen Broccoli, Landslide Creamy Pea..."
26532301,0.000264,"(Imagine Frozen Broccoli, Landslide Creamy Pea..."


time: 3min 16s (started: 2022-01-17 08:30:13 +00:00)


In [ ]:
transac_dec_rules = association_rules(transac_dec_freq_itemset, metric='confidence', min_threshold=0.05) # find rules